In [1]:
!pip install selenium

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
import time
import csv
import pandas as pd

In [ ]:
driver = webdriver.Chrome()

url_dic={
    'Andhra Pradesh':'https://www.redbus.in/online-booking/apsrtc',
    'Kerala':'https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile',
    'Telangana':'https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile',
    'Goa':'https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile',
    'Himachal Pradesh':'https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile',
    'Assam':'https://www.redbus.in/online-booking/astc/?utm_source=rtchometile',
    'Chandigarh':'https://www.redbus.in/online-booking/chandigarh-transport-undertaking-ctu',
    'Punjab':'https://www.redbus.in/online-booking/pepsu/?utm_source=rtchometile',
    'Bihar':'https://www.redbus.in/online-booking/bihar-state-road-transport-corporation-bsrtc/?utm_source=rtchometile',
    'West Bengal':'https://www.redbus.in/online-booking/west-bengal-transport-corporation?utm_source=rtchometile',
    'Sowth Bengal':'https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile',
    'Jammu Kashmir':'https://www.redbus.in/online-booking/jksrtc'
}

bus_data = []
try:
    for state, url in url_dic.items():
        driver.get(url)
        time.sleep(5)

        wait = WebDriverWait(driver, 10)
        page = 1

        all_routes = []
        
        while True:
            print(f"Processing page: {page}")
            try:
                nested_div = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[4]')))
                route_list = nested_div.find_elements(By.CLASS_NAME, 'route_link')  

                for route in route_list:
                    try:
                        link_element = route.find_element(By.TAG_NAME, 'a')
                        route_name = link_element.get_attribute('title')  
                        route_link = link_element.get_attribute('href')
                        all_routes.append((state, route_name, route_link))
                    except Exception as e:
                        print(f"Error extracting route data: {e}")

                page_buttons = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".DC_117_pageTabs")))

                if page >= len(page_buttons): 
                    print("Reached the last page.")
                    break

                next_button = page_buttons[page]
                driver.execute_script("arguments[0].click();", next_button)
                
                page+=1
                time.sleep(2)  

            except Exception as e:
                print(f"Error navigating data on page {page}: {e}")
                break

        for state, route_name, route_link in all_routes:
            try:
                driver.get(route_link)
                time.sleep(3)

                try:
                    view_buses_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".button")))
                    view_buses_button.click() 
                except Exception as e:
                            print(f"no govt busses for this route {e}")

                bus_details = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "bus-item")))
                
                for bus in bus_details:
                    try:
                        bus_name = bus.find_element(By.CLASS_NAME, "travels").text
                        bus_type = bus.find_element(By.CLASS_NAME, "bus-type").text
                        departure_time = bus.find_element(By.CLASS_NAME, "dp-time").text
                        duration = bus.find_element(By.CLASS_NAME, "dur").text
                        reaching_time = bus.find_element(By.CLASS_NAME, "bp-time").text
                        
                        try:
                            star_rating = bus.find_element(By.CSS_SELECTOR, ".rating-sec .rating span").text
                        except Exception:
                            star_rating  = "N/A"
                        
                        fare = bus.find_element(By.CLASS_NAME, "fare").text.split()[-1]  # Extract numeric fare
       
                        try:
                            seats_available = bus.find_element(By.CLASS_NAME, "seat-left").text.split()[0]
                        except Exception:
                            seats_available = "N/A"

                        bus_data.append({
                            "state": state,
                            "route_name": route_name,
                            "route_link": route_link,
                            "busname": bus_name,
                            "bustype": bus_type,
                            "departing_time": departure_time,
                            "duration": duration,
                            "reaching_time": reaching_time,
                            "star_rating": star_rating,
                            "price": fare,
                            "seats_available": seats_available
                            })
                    except:
                        pass

            except:
                pass

    with open("Redbus.csv", "w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=[
            "state", "route_name", "route_link", "busname", "bustype",
            "departing_time", "duration", "reaching_time", "star_rating", "price", "seats_available"
        ])
        writer.writeheader()
        writer.writerows(bus_data)
finally:
    driver.quit()

Processing page: 1
Processing page: 2
Processing page: 3
Processing page: 4
Processing page: 5
Reached the last page.
Error extracting bus details: list index out of range
Error extracting bus details: list index out of range
Error extracting bus details: list index out of range
Error extracting bus details: list index out of range
Error extracting bus details: list index out of range
Error extracting bus details: list index out of range
Error extracting bus details: list index out of range
Error extracting bus details: list index out of range
Error extracting bus details: list index out of range
Error extracting bus details: list index out of range
Error extracting bus details: list index out of range
Error extracting bus details: list index out of range
Error extracting bus details: list index out of range
Error extracting bus details: list index out of range
Error extracting bus details: list index out of range
Error extracting bus details: list index out of range
Error extracting b

In [21]:
import pandas as pd

In [22]:
data = pd.read_csv("Redbus.csv")

In [23]:
data['departing_time'] = pd.to_datetime(data['departing_time'], format='%H:%M', errors='coerce').dt.time
data['reaching_time'] = pd.to_datetime(data['reaching_time'], format='%H:%M', errors='coerce').dt.time

In [24]:
data.head()

,state,route_name,route_link,busname,bustype,departing_time,duration,reaching_time,star_rating,price,seats_available
0,Andhra Pradesh,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,IntrCity SmartBus,A/C Seater / Sleeper (2+1),23:05:00,06h 55m,06:00:00,4.6,689.0,20.0
1,Andhra Pradesh,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,IntrCity SmartBus,AC Sleeper (2+1),23:50:00,06h 10m,06:00:00,4.6,999.0,14.0
2,Andhra Pradesh,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,FRESHBUS,Electric A/C Seater (2+2),23:20:00,06h 50m,06:10:00,4.6,598.0,19.0
3,Andhra Pradesh,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,AdIntrCity SmartBus,NaN,23:05:00,06h 55m,06:00:00,4.5,489.0,26.0
4,Andhra Pradesh,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...,FRESHBUS,Electric A/C Seater (2+2),23:25:00,06h 10m,05:35:00,4.7,294.0,33.0


In [25]:
data.dropna(inplace=True)

In [26]:
import pymysql

In [27]:
mydb = pymysql.connect(
            host="localhost",
            user="root",
            password="1234",  # Update with your database password
            database = "Redbus_data" 
        )
mycursor = mydb.cursor()

In [28]:
mycursor.execute("show tables")


0

In [29]:
mycursor.execute("""CREATE TABLE buses (
    state VARCHAR(100),
    route_name VARCHAR(255),
    route_link TEXT,
    busname VARCHAR(255),
    bustype VARCHAR(255),
    departing_time TEXT,
    duration VARCHAR(50),
    reaching_time text,
    star_rating DECIMAL(2,1),
    price DECIMAL(10,2),
    seats_available INT
);""")
mydb.commit()

In [12]:
data.columns

Index(['state', 'route_name', 'route_link', 'busname', 'bustype',
       'departing_time', 'duration', 'reaching_time', 'star_rating', 'price',
       'seats_available'],
      dtype='object')

In [30]:
for i,row in data.iterrows():
    insert_query = """insert into buses(state,route_name,route_link,busname,bustype,departing_time,duration,reaching_time,
                       star_rating,price,seats_available)values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
    values = (row['state'],row['route_name'],row['route_link'],row['busname'],row['bustype'],row['departing_time'],row['duration'],
              row['reaching_time'],row['star_rating'],row['price'],row['seats_available'])
    mycursor.execute(insert_query,values)
    mydb.commit()

In [31]:
mycursor.execute("SELECT * FROM buses")
data = mycursor.fetchall()
columns = [desc[0] for desc in mycursor.description]
table = pd.DataFrame(data, columns=columns)

In [32]:
table

,state,route_name,route_link,busname,bustype,departing_time,duration,reaching_time,star_rating,price,seats_available
0,Andhra Pradesh,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,IntrCity SmartBus,A/C Seater / Sleeper (2+1),23:05:00,06h 55m,06:00:00,4.6,689.00,20
1,Andhra Pradesh,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,IntrCity SmartBus,AC Sleeper (2+1),23:50:00,06h 10m,06:00:00,4.6,999.00,14
2,Andhra Pradesh,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,FRESHBUS,Electric A/C Seater (2+2),23:20:00,06h 50m,06:10:00,4.6,598.00,19
3,Andhra Pradesh,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...,FRESHBUS,Electric A/C Seater (2+2),23:25:00,06h 10m,05:35:00,4.7,294.00,33
4,Andhra Pradesh,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...,FRESHBUS,Electric A/C Seater (2+2),16:30:00,06h 45m,23:15:00,4.7,294.00,27
...,...,...,...,...,...,...,...,...,...,...,...
1786,Jammu Kashmir,Jammu (j and k) to Haridwar,https://www.redbus.in/bus-tickets/jammu-to-har...,JKRTC - 178990,NON A/C Seater (2+3),17:30:00,12h 00m,05:30:00,3.5,828.00,42
1787,Jammu Kashmir,Jammu (j and k) to Haridwar,https://www.redbus.in/bus-tickets/jammu-to-har...,Laxmi holidays,Bharat Benz A/C Seater /Sleeper (2+1),20:50:00,12h 40m,09:30:00,4.4,628.00,22
1788,Jammu Kashmir,Jammu (j and k) to Haridwar,https://www.redbus.in/bus-tickets/jammu-to-har...,Laxmi holidays,Bharat Benz A/C Seater /Sleeper (2+1),17:45:00,12h 15m,06:00:00,4.3,630.00,10
1789,Jammu Kashmir,Jammu (j and k) to Haridwar,https://www.redbus.in/bus-tickets/jammu-to-har...,City Land Travels,A/C Seater/Sleeper (2+1),20:30:00,12h 30m,09:00:00,3.6,540.00,26


In [33]:
table.dtypes

state              object
route_name         object
route_link         object
busname            object
bustype            object
departing_time     object
duration           object
reaching_time      object
star_rating        object
price              object
seats_available     int64
dtype: object

In [29]:
statelist = table["state"].unique().tolist()
statelist

['Andhra Pradesh',
 'Kerala',
 'Telangana',
 'Goa',
 'Himachal Pradesh',
 'Assam',
 'Chandigarh',
 'Bihar',
 'West Bengal',
 'Sowth Bengal',
 'Jammu Kashmir']

In [31]:
routes = table[table["state"] == 'Andhra Pradesh']["route_name"].unique().tolist()
routes

['Hyderabad to Vijayawada',
 'Vijayawada to Hyderabad',
 'Bangalore to Tirupati',
 'Bangalore to Kadapa',
 'Kakinada to Visakhapatnam',
 'Ongole to Hyderabad',
 'Bangalore to Anantapur (andhra pradesh)',
 'Bangalore to Chittoor (Andhra Pradesh)',
 'Chittoor (Andhra Pradesh) to Bangalore',
 'Hyderabad to Guntur (Andhra Pradesh)',
 'Vijayawada to Visakhapatnam',
 'Hyderabad to Narasaraopet',
 'Kadapa to Bangalore',
 'Narasaraopet to Hyderabad',
 'Visakhapatnam to Kakinada',
 'Bangalore to Kadiri',
 'Bangalore to Madanapalli',
 'Madanapalli to Bangalore',
 'Anantapur (andhra pradesh) to Bangalore',
 'Hyderabad to Kurnool',
 'Hyderabad to Vinukonda',
 'Rajahmundry to Visakhapatnam',
 'Hyderabad to Eluru',
 'Guntur (Andhra Pradesh) to Hyderabad',
 'Hyderabad to Macherla (andhra pradesh)',
 'Bangalore to Rayachoti',
 'Hyderabad to Nandyal',
 'Eluru to Hyderabad',
 'Tirupati to Bangalore',
 'Vinukonda to Hyderabad',
 'Hyderabad to Anantapur (andhra pradesh)',
 'Hyderabad to Markapuram',
 'Ban

In [ ]:
filtered_buses = table[
                    (table["route_name"] == route_selected) &
                    (table["star_rating"] >= rating) &
                    (table["price"] <= max_price) &
                    (table["seats_available"] >= seats)
                ]

In [10]:
mydb.close()

NameError: name 'mydb' is not defined